In [ ]:
import keras

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

In [ ]:
import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Flatten, Lambda
from keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D
from keras.optimizers import RMSprop

In [ ]:
from keras import backend as K

In [ ]:
num_classes=10

In [ ]:
def create_pairs(x, digit_indices):
  pairs = []
  labels = []
  
  n=min([len(digit_indices[d]) for d in range(num_classes)]) -1
  
  for d in range(num_classes):
    for i in range(n):
      z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
      pairs += [[x[z1], x[z2]]]
      inc = random.randrange(1, num_classes)
      dn = (d + inc) % num_classes
      z1, z2 = digit_indices[d][i], digit_indices[dn][i]
      pairs += [[x[z1], x[z2]]]
      labels += [1,0]
  return np.array(pairs), np.array(labels)

In [ ]:
def euclid_dis(x_y):
  x,y = x_y
  sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
  
def regularize_cross_entropy(p, q, ets=1e-15):
	return -sum([p[i]*log(q[i]+ets) for i in range(len(p))])

In [ ]:
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

input_shape = x_train.shape[1:]

In [ ]:
def siamese_model(input_shape):
  
  input = Input(shape = input_shape)
  x = Conv2D(32, (3,3), activation = 'relu')(input)
  x = AveragePooling2D(pool_size = (2,2))(x)
  x = Conv2D(64, (3,3), activation = 'tanh')(x)
  x = MaxPooling2D(pool_size = (2,2))(x)
  x = Dropout(0.25)(x)
  x = Flatten()(x)
  x = Dense(128, activation = 'tanh')(x)
  x = Dropout(0.5)(x)
  x = Dense(64,activation = 'tanh')(x)
  x = Dropout(0.5)(x)
  x = Dense(10,activation = 'tanh')(x)
  model = Model(input, x)
  model.summary()
  return model


In [ ]:
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
pairs_train, y_train = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
pairs_test, y_test = create_pairs(x_test, digit_indices)

base_network = siamese_model(input_shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0     

In [ ]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclid_dis, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

In [ ]:
y_test = np.array(y_test, dtype='float32')
y_train = np.array(y_train, dtype='float32')

In [ ]:
def fit_model(optim, n_epochs, loss_func):
    model = Model([input_a, input_b], distance)   
    model.compile(loss=loss_func, optimizer=optim, metrics=[accuracy])
    model.fit([pairs_train[:, 0], pairs_train[:, 1]], y_train, batch_size=128, epochs=n_epochs, validation_data=([pairs_test[:, 0], pairs_test[:, 1]], y_test))
    return model

In [ ]:
def predict(model):
    y_pred = model.predict([pairs_train[:, 0], pairs_train[:, 1]])
    tr_acc = compute_accuracy(y_train, y_pred)
    y_pred = model.predict([pairs_test[:, 0], pairs_test[:, 1]])
    te_acc = compute_accuracy(y_test, y_pred)

    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))
    return te_acc

**Trying different loss functions**

In [ ]:
# LOSS FUNCTION : "regularized_cross_entropy"
opt = 'adam'
n_epochs = 2
regularized_cross_entropy = keras.losses.BinaryCrossentropy()
siam_model = fit_model(opt, n_epochs, regularized_cross_entropy)
test_accuracy_adm = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 153s 179ms/step - loss: 0.8917 - accuracy: 0.5185 - val_loss: 3.1775 - val_accuracy: 0.5000
Epoch 2/2
847/847 [==============================] - 151s 179ms/step - loss: 0.7258 - accuracy: 0.5011 - val_loss: 2.5741 - val_accuracy: 0.5000
* Accuracy on training set: 50.00%
* Accuracy on test set: 50.00%


In [ ]:
# LOSS FUNCTION : "contrastive_loss"
opt = 'adam'
n_epochs = 2
siam_model = fit_model(opt, n_epochs, contrastive_loss)
test_accuracy_adm = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 153s 179ms/step - loss: 0.1418 - accuracy: 0.7870 - val_loss: 0.0376 - val_accuracy: 0.9541
Epoch 2/2
847/847 [==============================] - 155s 183ms/step - loss: 0.0604 - accuracy: 0.9509 - val_loss: 0.0301 - val_accuracy: 0.9679
* Accuracy on training set: 97.29%
* Accuracy on test set: 96.77%


**Trying different optimizers**

In [ ]:
# optimizer : ADAM
opt = 'adam'
n_epochs = 2
siam_model = fit_model(opt, n_epochs, contrastive_loss)
test_accuracy_adm = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 151s 176ms/step - loss: 0.1814 - accuracy: 0.7328 - val_loss: 0.0719 - val_accuracy: 0.9289
Epoch 2/2
847/847 [==============================] - 155s 183ms/step - loss: 0.1046 - accuracy: 0.8845 - val_loss: 0.0641 - val_accuracy: 0.9448
* Accuracy on training set: 94.72%
* Accuracy on test set: 94.45%


In [ ]:
# optimizer : RMSprop
opt = 'RMSprop'
n_epochs = 2
siam_model = fit_model(opt, n_epochs, contrastive_loss)
test_accuracy_rmsprop = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 150s 176ms/step - loss: 0.0933 - accuracy: 0.9040 - val_loss: 0.0553 - val_accuracy: 0.9492
Epoch 2/2
847/847 [==============================] - 152s 180ms/step - loss: 0.0830 - accuracy: 0.9213 - val_loss: 0.0508 - val_accuracy: 0.9577
* Accuracy on training set: 95.96%
* Accuracy on test set: 95.75%


In [ ]:
# optimizer : Mini-Batch Gradient Descent
opt = 'SGD' # batch-size is given in "fit" method
n_epochs = 2
siam_model = fit_model(opt, n_epochs, contrastive_loss)
test_accuracy_mbgd = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 155s 182ms/step - loss: 0.0753 - accuracy: 0.9348 - val_loss: 0.0511 - val_accuracy: 0.9613
Epoch 2/2
847/847 [==============================] - 151s 179ms/step - loss: 0.0738 - accuracy: 0.9374 - val_loss: 0.0515 - val_accuracy: 0.9617
* Accuracy on training set: 96.54%
* Accuracy on test set: 96.14%


In [ ]:
import pandas as pd
data = [['ADAM', test_accuracy_adm], ['RMSProp', test_accuracy_rmsprop], ['Mini-Batch Gradient Descent', test_accuracy_mbgd]]
df = pd.DataFrame(data, columns = ['Optimizer', 'Accuracy'])
# print dataframe.
print(df)

                     Optimizer  Accuracy
0                         ADAM  0.944501
1                      RMSProp  0.957464
2  Mini-Batch Gradient Descent  0.961448


**REASON :**

We see that **"Adam" optimizer performs relatively better than "RMSProp" and "Mini-Batch Gradient Descent"**. This is because Adam (adaptive moment estimation), also uses past learning rates like AdaGrad and RMSProp do.
Adam is a replacement optimization algorithm for stochastic gradient descent for training deep learning models.
Adam is relatively easy to configure where the default configuration parameters do well on most problems.

**Hyper Parameter Optimization** 

---
Trying to improve the accuracy of "Mini-Batch Gradient Descent" model by :


*   Increasing epochs
*   Using learning rate
*   Using decay rate
*   Using momentum



In [ ]:
# optimizer : Mini-Batch Gradient Descent
epochs=2
learning_rate = 0.02
decay_rate = learning_rate / epochs
momentum = 0.8

opt = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
siam_model = fit_model(opt, epochs, contrastive_loss)
test_accuracy_mbgd = predict(siam_model)

Epoch 1/2
847/847 [==============================] - 154s 180ms/step - loss: 0.1122 - accuracy: 0.8632 - val_loss: 0.0925 - val_accuracy: 0.8847
Epoch 2/2
847/847 [==============================] - 153s 180ms/step - loss: 0.1081 - accuracy: 0.8696 - val_loss: 0.0923 - val_accuracy: 0.8850
* Accuracy on training set: 88.39%
* Accuracy on test set: 88.44%


In [ ]:
# optimizer : Mini-Batch Gradient Descent
epochs=10
learning_rate = 0.04
decay_rate = learning_rate / (1.5*epochs)
momentum = 0.8

opt = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
siam_model = fit_model(opt, epochs, contrastive_loss)
test_accuracy_mbgd = predict(siam_model)

Epoch 1/10
847/847 [==============================] - 153s 180ms/step - loss: 0.0595 - accuracy: 0.9553 - val_loss: 0.0278 - val_accuracy: 0.9596
Epoch 2/10
847/847 [==============================] - 152s 180ms/step - loss: 0.0540 - accuracy: 0.9624 - val_loss: 0.0265 - val_accuracy: 0.9721
Epoch 3/10
847/847 [==============================] - 152s 180ms/step - loss: 0.0523 - accuracy: 0.9634 - val_loss: 0.0289 - val_accuracy: 0.9659
Epoch 4/10
847/847 [==============================] - 154s 182ms/step - loss: 0.0519 - accuracy: 0.9642 - val_loss: 0.0262 - val_accuracy: 0.9686
Epoch 5/10
847/847 [==============================] - 153s 180ms/step - loss: 0.0516 - accuracy: 0.9655 - val_loss: 0.0264 - val_accuracy: 0.9690
Epoch 6/10
847/847 [==============================] - 153s 181ms/step - loss: 0.0515 - accuracy: 0.9649 - val_loss: 0.0279 - val_accuracy: 0.9689
Epoch 7/10
847/847 [==============================] - 154s 181ms/step - loss: 0.0518 - accuracy: 0.9649 - val_loss: 0.0276 -

# ***Improved Accuracy***
**We observe that the accuracy in test data increased from 96.1448% to 97.49%**

**Pros and Cons of Siamese Networks**

PROS :

*More Robust to class Imbalance*: With the aid of One-shot learning, given a few images per class is sufficient for Siamese Networks to recognize those images in the future

*Nice to an ensemble with the best classifier*: Given that its learning mechanism is somewhat different from Classification, simple averaging of it with a Classifier can do much better than average 2 correlated Supervised models (e.g. GBM & RF classifier)

*Learning from Semantic Similarity*: Siamese focuses on learning embeddings (in the deeper layer) that place the same classes/concepts close together. Hence, can learn semantic similarity.

**CONS:**

*Needs more training time than normal networks:* Since Siamese Networks involves quadratic pairs to learn from (to see all information available) it is slower than normal classification type of learning(pointwise learning)

Doesn’t output probabilities: Since training involves pairwise learning, it won’t output the probabilities of the prediction, but the distance from each class